In [3]:
import pickle as pk
import tensorflow as tf
import numpy as np
import tensorflow as tf
from tensorflow.keras import regularizers

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

import os, sys, pathlib, h5py, time

from models import make_model

from astropy.nddata import block_reduce

from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2022-08-22 10:51:42.491948: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 17403 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0004:04:00.0, compute capability: 7.0
2022-08-22 10:51:42.494024: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 17407 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB, pci bus id: 0004:05:00.0, compute capability: 7.0
2022-08-22 10:51:42.496494: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 31016 MB memory:  -> device: 2, name: Tesla V100-SXM2-32GB, pci bus id: 0035:03:00.0, compute capability: 7.0
2022-08-22 10:51:42.498923: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 31016 MB memory:  -> device: 3, name: Tesla V100-SXM2-32GB, pci bus id

In [4]:
data_dir = '/project/r/rbond/jorlo/datasets/ml-clusters/for-debug/'

w_y = 'True'

cut = 10000

In [5]:
pos_im_act = []
pos_im_des = []
neg_im = []

for directory in os.listdir(data_dir):
    print(directory)
    if directory[:3] == 'act' and (int(directory[4:8]) < cut):
        h5f = h5py.File(data_dir+directory)
        pos_im_act.append(h5f['act'][:])
    elif directory[:3] == 'des' and (int(directory[4:8]) < cut):
        h5f = h5py.File(data_dir+directory)
        pos_im_des.append(h5f['des'][:])
    elif directory[:6]=='random' and (int(directory[7:11]) < cut):
        h5f = h5py.File(data_dir+directory)
        neg_im.append(h5f['random'][:])


pos_im_act = np.vstack(pos_im_act)
pos_im_des = np.vstack(pos_im_des)
neg_im = np.vstack(neg_im)


pos_im = np.concatenate((pos_im_act, pos_im_des))


flags = []
for i in range(pos_im.shape[0]):
        if np.any(np.isnan(pos_im[i,...])):
                flags.append(i)

pos_im = np.delete(pos_im, flags, axis = 0)

flags = []
for i in range(neg_im.shape[0]):
        if np.any(np.isnan(neg_im[i,...])):
                flags.append(i)

neg_im = np.delete(neg_im, flags, axis = 0)


neg_im = neg_im[:len(pos_im)]
pos_im = pos_im[:len(neg_im)]

print(len(pos_im), len(neg_im))

save_neg_im = neg_im
save_pos_im = pos_im

random_2000_w_y.h5
des_5000_w_y.h5
des_2000_w_y.h5
random_0000_w_y.h5
des_0000_w_y.h5
des_1000_w_y.h5
random_6000_w_y.h5
random_7000_w_y.h5
random_5000_w_y.h5
des_6000_w_y.h5
act_1000_w_y.h5
random_4000_w_y.h5
random_3000_w_y.h5
act_3000_w_y.h5
des_4000_w_y.h5
random_1000_w_y.h5
act_0000_w_y.h5
des_3000_w_y.h5
5918 5918


In [6]:


reduce = 8

if reduce:
    neg_im_2 = []
    pos_im_2 = []

    for i in range(len(neg_im)):
        neg_im_2.append(block_reduce(save_neg_im[i], [reduce,reduce,1], func = np.mean))
        pos_im_2.append(block_reduce(save_pos_im[i], [reduce,reduce,1], func = np.mean))
        

    pos_im = np.array(pos_im_2)
    neg_im = np.array(neg_im_2)


In [7]:
cut_2 = 1000
if cut_2:
    neg_im = neg_im[:cut_2]
    pos_im = pos_im[:cut_2]

    
print(neg_im.shape)

(1000, 49, 49, 6)


In [8]:
if w_y == 'False':
    pos_im, neg_im = pos_im[...,:5], neg_im[...,:5]

print('Fitting with y = ', w_y)

tot = min(pos_im.shape[0], neg_im.shape[0])
train_size = int(0.7 * tot)
val_size = int(0.15 * tot)
test_size = int(0.15 * tot)

train_pos = pos_im[:train_size]
val_pos = pos_im[train_size:train_size + val_size]
test_pos = pos_im[train_size + val_size:]

train_neg = neg_im[:train_size]
val_neg = neg_im[train_size:train_size + val_size]
test_neg = neg_im[train_size + val_size:]

input_shape = train_pos.shape[1:]

train_images = np.concatenate((train_pos,train_neg))
val_images = np.concatenate((val_pos,val_neg))
test_images = np.concatenate((test_pos,test_neg))

train_labels = np.array(train_pos.shape[0]*[1] + train_neg.shape[0]*[0])
val_labels = np.array(val_pos.shape[0]*[1] + val_neg.shape[0]*[0])
test_labels = np.array(test_pos.shape[0]*[1] + test_neg.shape[0]*[0])

batch_size = 500
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)
val_dataset = val_dataset.shuffle(buffer_size=1024).batch(batch_size)
test_dataset = test_dataset.shuffle(buffer_size=1024).batch(batch_size)

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)


Fitting with y =  True


2022-08-22 10:55:42.268777: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 17403 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0004:04:00.0, compute capability: 7.0
2022-08-22 10:55:42.270511: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 17407 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB, pci bus id: 0004:05:00.0, compute capability: 7.0
2022-08-22 10:55:42.274204: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 31016 MB memory:  -> device: 2, name: Tesla V100-SXM2-32GB, pci bus id: 0035:03:00.0, compute capability: 7.0
2022-08-22 10:55:42.276464: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 31016 MB memory:  -> device: 3, name: Tesla V100-SXM2-32GB, pci bus id

In [9]:
model = make_model('test', input_shape = input_shape, kernel_regularizer=regularizers.l2(0.001))


data_augmentation = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=360, width_shift_range=4,
    height_shift_range=4,zoom_range=0.3)

if w_y == 'False':
        checkpoint_path = "/scratch/r/rbond/jorlo/ml-clusters/models/jupyter_redmapper_wo_y.ckpt"

else:
        checkpoint_path = "/scratch/r/rbond/jorlo/ml-clusters/models/jupyter_redmapper_w_y.ckpt"

        
checkpoint_dir = os.path.dirname(checkpoint_path)

def get_callbacks(name):
    return [tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1),
      tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=100),
      tf.keras.callbacks.TensorBoard(checkpoint_dir+name),
    ]        


# Create a callback that saves the model's weights

name = 'w_y={}'.format(w_y)

toc = time.time()
history = model.fit(data_augmentation.flow(train_images, train_labels), epochs=int(50), 
                    validation_data=val_dataset, callbacks=get_callbacks(name))
tic = time.time()

print('Distributed time: ', tic-toc)
test_loss, test_acc = model.evaluate(test_dataset, verbose=2)



INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Number of devices: 4
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 49, 49, 16)        880       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 16)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 24, 24, 16)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        4640      
                                                      

/home/r/rbond/jorlo/.conda/envs/ml-clusters/lib/python3.8/site-packages/keras_preprocessing/image/numpy_array_iterator.py:129: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (1400, 49, 49, 6) (6 channels).
  warnings.warn('NumpyArrayIterator is set to use the '
2022-08-22 10:55:44.690627: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_2"
op: "FlatMapDataset"
input: "TensorDataset/_1"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_flat_map_fn_517"


Epoch 1/100
INFO:tensorflow:batch_all_reduce: 10 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:batch_all_reduce: 10 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/

2022-08-22 10:56:01.006652: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101
2022-08-22 10:56:07.049141: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101
2022-08-22 10:56:12.343245: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101
2022-08-22 10:56:17.359912: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101
2022-08-22 10:56:18.564132: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-08-22 10:56:18.564853: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-08-22 10:56:18.564883: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-08-22 10:56:18.565498: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or direct

44/44 [==============================] - ETA: 0s - loss: 1.0179 - accuracy: 0.8757

...

Epoch 27/100
44/44 [==============================] - ETA: 0s - loss: 0.1258 - accuracy: 0.9707
Epoch 27: saving model to /scratch/r/rbond/jorlo/ml-clusters/models/jupyter_redmapper_w_y.ckpt
44/44 [==============================] - 3s 61ms/step - loss: 0.1258 - accuracy: 0.9707 - val_loss: 0.1244 - val_accuracy: 0.9767
Epoch 28/100
43/44 [============================>.] - ETA: 0s - loss: 0.1311 - accuracy: 0.9737
Epoch 28: saving model to /scratch/r/rbond/jorlo/ml-clusters/models/jupyter_redmapper_w_y.ckpt
44/44 [==============================] - 3s 63ms/step - loss: 0.1308 - accuracy: 0.9729 - val_loss: 0.1390 - val_accuracy: 0.9700
Epoch 29/100
43/44 [============================>.] - ETA: 0s - loss: 0.1389 - accuracy: 0.9671
Epoch 29: saving model to /scratch/r/rbond/jorlo/ml-clusters/models/jupyter_redmapper_w_y.ckpt
44/44 [==============================] - 3s 65ms/step - loss: 0.1370 - accuracy: 0.9679 - val_loss: 0.1327 - val_accuracy: 0.9733
Epoch 30/100
44/44 [=================

44/44 [==============================] - 3s 63ms/step - loss: 0.1195 - accuracy: 0.9714 - val_loss: 0.1079 - val_accuracy: 0.9700
Epoch 53/100
44/44 [==============================] - ETA: 0s - loss: 0.1142 - accuracy: 0.9736
Epoch 53: saving model to /scratch/r/rbond/jorlo/ml-clusters/models/jupyter_redmapper_w_y.ckpt
44/44 [==============================] - 3s 67ms/step - loss: 0.1142 - accuracy: 0.9736 - val_loss: 0.1091 - val_accuracy: 0.9733
Epoch 54/100
43/44 [============================>.] - ETA: 0s - loss: 0.1104 - accuracy: 0.9693
Epoch 54: saving model to /scratch/r/rbond/jorlo/ml-clusters/models/jupyter_redmapper_w_y.ckpt
44/44 [==============================] - 3s 60ms/step - loss: 0.1109 - accuracy: 0.9686 - val_loss: 0.1143 - val_accuracy: 0.9700
Epoch 55/100
44/44 [==============================] - ETA: 0s - loss: 0.0982 - accuracy: 0.9750
Epoch 55: saving model to /scratch/r/rbond/jorlo/ml-clusters/models/jupyter_redmapper_w_y.ckpt
44/44 [==============================

43/44 [============================>.] - ETA: 0s - loss: 0.1102 - accuracy: 0.9656
Epoch 78: saving model to /scratch/r/rbond/jorlo/ml-clusters/models/jupyter_redmapper_w_y.ckpt
44/44 [==============================] - 3s 57ms/step - loss: 0.1089 - accuracy: 0.9664 - val_loss: 0.1132 - val_accuracy: 0.9700
Epoch 79/100
43/44 [============================>.] - ETA: 0s - loss: 0.0976 - accuracy: 0.9773
Epoch 79: saving model to /scratch/r/rbond/jorlo/ml-clusters/models/jupyter_redmapper_w_y.ckpt
44/44 [==============================] - 3s 62ms/step - loss: 0.0978 - accuracy: 0.9771 - val_loss: 0.1103 - val_accuracy: 0.9700
Epoch 80/100
43/44 [============================>.] - ETA: 0s - loss: 0.1042 - accuracy: 0.9715
Epoch 80: saving model to /scratch/r/rbond/jorlo/ml-clusters/models/jupyter_redmapper_w_y.ckpt
44/44 [==============================] - 3s 66ms/step - loss: 0.1028 - accuracy: 0.9721 - val_loss: 0.1314 - val_accuracy: 0.9700
Epoch 81/100
43/44 [============================>.

2022-08-22 11:01:53.553176: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_DOUBLE
      type: DT_INT64
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 300
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024TensorSliceDataset:2"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 49
        }
        dim {
          size: 49
        }
        dim {
          size: 6
        }
      }
      shape {
      }
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUC

1/1 - 0s - loss: 0.1145 - accuracy: 0.9633 - 499ms/epoch - 499ms/step


In [10]:
model = make_model('test-no-dist', input_shape = input_shape, kernel_regularizer=regularizers.l2(0.001))


data_augmentation = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=360, width_shift_range=4,
    height_shift_range=4,zoom_range=0.3)

if w_y == 'False':
        checkpoint_path = "/scratch/r/rbond/jorlo/ml-clusters/models/jupyter_redmapper_wo_y.ckpt"

else:
        checkpoint_path = "/scratch/r/rbond/jorlo/ml-clusters/models/jupyter_redmapper_w_y.ckpt"

        
checkpoint_dir = os.path.dirname(checkpoint_path)

def get_callbacks(name):
    return [tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1),
      tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=100),
      tf.keras.callbacks.TensorBoard(checkpoint_dir+name),
    ]        


# Create a callback that saves the model's weights

name = 'w_y={}'.format(w_y)

toc = time.time()
history = model.fit(data_augmentation.flow(train_images, train_labels), epochs=int(100), 
                    validation_data=val_dataset, callbacks=get_callbacks(name))
tic = time.time()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Number of devices: 4
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 49, 49, 16)        880       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 24, 24, 16)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 24, 24, 16)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 24, 24, 32)        4640      
                                                    

44/44 [==============================] - ETA: 0s - loss: 0.1602 - accuracy: 0.9671
Epoch 19: saving model to /scratch/r/rbond/jorlo/ml-clusters/models/jupyter_redmapper_w_y.ckpt
44/44 [==============================] - 3s 57ms/step - loss: 0.1602 - accuracy: 0.9671 - val_loss: 0.1632 - val_accuracy: 0.9567
Epoch 20/100
44/44 [==============================] - ETA: 0s - loss: 0.1547 - accuracy: 0.9686
Epoch 20: saving model to /scratch/r/rbond/jorlo/ml-clusters/models/jupyter_redmapper_w_y.ckpt
44/44 [==============================] - 2s 50ms/step - loss: 0.1547 - accuracy: 0.9686 - val_loss: 0.1516 - val_accuracy: 0.9633
Epoch 21/100
44/44 [==============================] - ETA: 0s - loss: 0.1448 - accuracy: 0.9657
Epoch 21: saving model to /scratch/r/rbond/jorlo/ml-clusters/models/jupyter_redmapper_w_y.ckpt
44/44 [==============================] - 9s 203ms/step - loss: 0.1448 - accuracy: 0.9657 - val_loss: 0.1774 - val_accuracy: 0.9500
Epoch 22/100
44/44 [=============================

44/44 [==============================] - 2s 52ms/step - loss: 0.1266 - accuracy: 0.9671 - val_loss: 0.1307 - val_accuracy: 0.9633
Epoch 45/100
43/44 [============================>.] - ETA: 0s - loss: 0.1072 - accuracy: 0.9759
Epoch 45: saving model to /scratch/r/rbond/jorlo/ml-clusters/models/jupyter_redmapper_w_y.ckpt
44/44 [==============================] - 2s 50ms/step - loss: 0.1058 - accuracy: 0.9764 - val_loss: 0.1447 - val_accuracy: 0.9667
Epoch 46/100
44/44 [==============================] - ETA: 0s - loss: 0.1160 - accuracy: 0.9686
Epoch 46: saving model to /scratch/r/rbond/jorlo/ml-clusters/models/jupyter_redmapper_w_y.ckpt
44/44 [==============================] - 2s 54ms/step - loss: 0.1160 - accuracy: 0.9686 - val_loss: 0.1623 - val_accuracy: 0.9567
Epoch 47/100
44/44 [==============================] - ETA: 0s - loss: 0.1132 - accuracy: 0.9764
Epoch 47: saving model to /scratch/r/rbond/jorlo/ml-clusters/models/jupyter_redmapper_w_y.ckpt
44/44 [==============================

44/44 [==============================] - ETA: 0s - loss: 0.1033 - accuracy: 0.9707
Epoch 70: saving model to /scratch/r/rbond/jorlo/ml-clusters/models/jupyter_redmapper_w_y.ckpt
44/44 [==============================] - 2s 48ms/step - loss: 0.1033 - accuracy: 0.9707 - val_loss: 0.1107 - val_accuracy: 0.9700
Epoch 71/100
44/44 [==============================] - ETA: 0s - loss: 0.1026 - accuracy: 0.9693
Epoch 71: saving model to /scratch/r/rbond/jorlo/ml-clusters/models/jupyter_redmapper_w_y.ckpt
44/44 [==============================] - 2s 48ms/step - loss: 0.1026 - accuracy: 0.9693 - val_loss: 0.1624 - val_accuracy: 0.9500
Epoch 72/100
43/44 [============================>.] - ETA: 0s - loss: 0.1164 - accuracy: 0.9678
Epoch 72: saving model to /scratch/r/rbond/jorlo/ml-clusters/models/jupyter_redmapper_w_y.ckpt
44/44 [==============================] - 2s 46ms/step - loss: 0.1149 - accuracy: 0.9686 - val_loss: 0.1115 - val_accuracy: 0.9733
Epoch 73/100
43/44 [============================>.

44/44 [==============================] - 2s 48ms/step - loss: 0.1007 - accuracy: 0.9736 - val_loss: 0.1165 - val_accuracy: 0.9633
Epoch 96/100
43/44 [============================>.] - ETA: 0s - loss: 0.0941 - accuracy: 0.9759
Epoch 96: saving model to /scratch/r/rbond/jorlo/ml-clusters/models/jupyter_redmapper_w_y.ckpt
44/44 [==============================] - 2s 48ms/step - loss: 0.0926 - accuracy: 0.9764 - val_loss: 0.1880 - val_accuracy: 0.9467
Epoch 97/100
43/44 [============================>.] - ETA: 0s - loss: 0.0987 - accuracy: 0.9708
Epoch 97: saving model to /scratch/r/rbond/jorlo/ml-clusters/models/jupyter_redmapper_w_y.ckpt
44/44 [==============================] - 2s 50ms/step - loss: 0.0975 - accuracy: 0.9714 - val_loss: 0.0930 - val_accuracy: 0.9800
Epoch 98/100
44/44 [==============================] - ETA: 0s - loss: 0.1130 - accuracy: 0.9700
Epoch 98: saving model to /scratch/r/rbond/jorlo/ml-clusters/models/jupyter_redmapper_w_y.ckpt
44/44 [==============================

In [11]:
print('Undistributed time: ', tic-toc)


Undistributed time:  247.986478805542


In [23]:
test_loss, test_acc = model.evaluate(test_dataset, verbose=2)

print('\nTest accuracy: ', test_acc)

#Turn off Y info

if w_y == 'True':
    no_y_model = make_model('test', input_shape = input_shape)
    
    checkpoint_path = "/scratch/r/rbond/jorlo/ml-clusters/models/jupyter_redmapper_w_y.ckpt"


    no_y_model.load_weights(checkpoint_path)

    #I think this is zeroing everything but y?
    for layer in no_y_model.layers:
        if layer.name[0:6] == 'conv2d':
            temp = layer.get_weights()
            #temp[0][...,-1:,:] = np.zeros(temp[0][...,-1:,:].shape)
            layer.set_weights(temp)
            for j in range(temp.shape[-1]):
                plt.imshow(temp[...:j])
                plt.show()
                plt.close()

    test_loss, test_acc = no_y_model.evaluate(test_dataset, verbose=2)

    print('\nNo y test accuracy: ', test_acc)
        

1/1 - 0s - loss: 0.4790 - accuracy: 0.7933 - 53ms/epoch - 53ms/step

Test accuracy:  0.7933333516120911


In [ ]:
show_compressed = True

for i in range(5):
    print(i)
    if show_compressed:
        if w_y == 'True':
            plt.imshow(neg_im[i,...,5])
            plt.title('Negative in y')
            plt.show()
            plt.close()


        plt.imshow(np.log(np.abs(neg_im[i,...,1])))
        plt.title('Negative in Optical, r band')
        plt.show()
        plt.close()


        if w_y == 'True':
            plt.imshow(pos_im[i,...,5])
            plt.title('Positive in y')
            plt.show()
            plt.close()


        plt.imshow(np.log(np.abs(pos_im[i,...,1])))
        plt.title('Positive in Optical, r band')
        plt.show()
        plt.close()

    else:
        if w_y == 'True':
            plt.imshow(save_neg_im[i,...,5])
            plt.title('Negative in y')
            plt.show()
            plt.close()


        plt.imshow(np.log(np.abs(save_neg_im[i,...,1])))
        plt.title('Negative in Optical, r band')
        plt.show()
        plt.close()


        if w_y == 'True':
            plt.imshow(save_pos_im[i,...,5])
            plt.title('Positive in y')
            plt.show()
            plt.close()


        plt.imshow(np.log(np.abs(save_pos_im[i,...,1])))
        plt.title('Positive in Optical, r band')
        plt.show()
        plt.close()


In [ ]:
#View Model weights

labels = ['g','r','i','z','y','comptony']

for layer in model.layers:
    if layer.name[0:6] == 'conv2d':
        temp = layer.get_weights()
        print('Layer: ',layer.name)
        print(temp[0].shape)
        for j in range(temp[0].shape[-1]):
            print('Filter: ',j)
            for k in range(temp[0][...,j].shape[-1]):
                print('Band: ', labels[k])
                plt.imshow(temp[0][...,j][...,k], vmin = np.amin(temp[0]), vmax = np.amax(temp[0]))
                plt.colorbar()
                plt.show()
                plt.close()

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset, verbose=2)


In [ ]:
model = make_model('test',  input_shape = input_shape)

checkpoint_path = "/scratch/r/rbond/jorlo/ml-clusters/models/redmapper_wo_y.ckpt"
model.load_weights(checkpoint_path)

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset, verbose=2)


# Hyperparameters

In [16]:
tuner = make_model('test_hyper')

In [17]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20)

tuner.search(train_images, train_labels, epochs=200, validation_data=val_dataset, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('dropout1')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


Trial 100 Complete [00h 00m 09s]
val_accuracy: 0.9766666889190674

Best val_accuracy So Far: 0.996666669845581
Total elapsed time: 00h 23m 47s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 0.5 and the optimal learning rate for the optimizer
is 0.0001.



In [18]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_images, train_labels, epochs=50, validation_data=val_dataset)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
44/44 [==============================] - 1s 10ms/step - loss: 0.3276 - accuracy: 0.8471 - val_loss: 0.1702 - val_accuracy: 0.9233
Epoch 2/50
44/44 [==============================] - 0s 7ms/step - loss: 0.0960 - accuracy: 0.9593 - val_loss: 0.1301 - val_accuracy: 0.9500
Epoch 3/50
44/44 [==============================] - 0s 7ms/step - loss: 0.0714 - accuracy: 0.9736 - val_loss: 0.1108 - val_accuracy: 0.9567
Epoch 4/50
44/44 [==============================] - 0s 6ms/step - loss: 0.0670 - accuracy: 0.9779 - val_loss: 0.0629 - val_accuracy: 0.9733
Epoch 5/50
44/44 [==============================] - 0s 6ms/step - loss: 0.0696 - accuracy: 0.9736 - val_loss: 0.0668 - val_accuracy: 0.9733
Epoch 6/50
44/44 [==============================] - 0s 6ms/step - loss: 0.0520 - accuracy: 0.9850 - val_loss: 0.0583 - val_accuracy: 0.9833
Epoch 7/50
44/44 [==============================] - 0s 7ms/step - loss: 0.0532 - accuracy: 0.9821 - val_loss: 0.0548 - val_accuracy: 0.9733
Epoch 8/50
44/44 [=

In [19]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(train_images, train_labels, epochs=best_epoch, validation_data=val_dataset)

Epoch 1/25
44/44 [==============================] - 1s 10ms/step - loss: 0.2724 - accuracy: 0.8914 - val_loss: 0.1415 - val_accuracy: 0.9367
Epoch 2/25
44/44 [==============================] - 0s 6ms/step - loss: 0.0831 - accuracy: 0.9693 - val_loss: 0.1430 - val_accuracy: 0.9467
Epoch 3/25
44/44 [==============================] - 0s 6ms/step - loss: 0.0714 - accuracy: 0.9721 - val_loss: 0.0984 - val_accuracy: 0.9667
Epoch 4/25
44/44 [==============================] - 0s 7ms/step - loss: 0.0645 - accuracy: 0.9750 - val_loss: 0.0879 - val_accuracy: 0.9667
Epoch 5/25
44/44 [==============================] - 0s 6ms/step - loss: 0.0588 - accuracy: 0.9807 - val_loss: 0.0537 - val_accuracy: 0.9800
Epoch 6/25
44/44 [==============================] - 0s 6ms/step - loss: 0.0561 - accuracy: 0.9821 - val_loss: 0.0527 - val_accuracy: 0.9833
Epoch 7/25
44/44 [==============================] - 0s 7ms/step - loss: 0.0528 - accuracy: 0.9843 - val_loss: 0.0567 - val_accuracy: 0.9800
Epoch 8/25
44/44 [=

In [21]:
test_loss, test_acc = model.evaluate(test_dataset, verbose=2)

print('\nTest accuracy: ', test_acc)

#Turn off Y info

if w_y == 'True':
    no_y_model = make_model('test', input_shape = input_shape)
    
    checkpoint_path = "/scratch/r/rbond/jorlo/ml-clusters/results/hyper_parameters/jupyter_redmapper_w_y.ckpt"


    no_y_model.load_weights(checkpoint_path)

    #I think this is zeroing everything but y?
    for layer in no_y_model.layers:
        if layer.name[0:6] == 'conv2d':
            temp = layer.get_weights()
            #temp[0][...,-1:,:] = np.zeros(temp[0][...,-1:,:].shape)
            layer.set_weights(temp)
            for j in range(temp.shape[-1]):
                plt.imshow(temp[...:j])
                plt.show()
                plt.close()

    test_loss, test_acc = no_y_model.evaluate(test_dataset, verbose=2)

    print('\nNo y test accuracy: ', test_acc)
        

1/1 - 0s - loss: 0.0641 - accuracy: 0.9867 - 59ms/epoch - 59ms/step

Test accuracy:  0.9866666793823242
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 49, 49, 16)        880       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 24, 24, 16)       0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 24, 24, 16)        0         
                                                                 
 conv2d_10 (Conv2D)          (None, 24, 24, 32)        4640      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 12, 12, 32)       0         
 g2D)                                                            
                

ValueError: Shapes (2304, 512) and (36864, 512) are incompatible